In [6]:
import tensorflow as tf
import os
import io
from PIL import Image
import xml.etree.ElementTree as ET
import random

from object_detection.utils import dataset_util

In [7]:
folder = 'train222/a'
images_all = os.listdir(folder)

In [8]:
images_all = [img for img in images_all if 'xml' not in img]

In [10]:
images_all = [img for img in images_all if 'xml' in img or 'png' in img or 'jpg' in img]

In [11]:
len(images_all)

750

In [12]:
random.shuffle(images_all)

In [19]:
num = int(len(images_all) * 0.85)
num

637

In [15]:
train_images = images_all[:num]
test_images = images_all[num:]

In [16]:
len(train_images), len(test_images)

(637, 113)

In [17]:
def get_coords(data_file):
    tree = ET.parse(data_file)
    root = tree.getroot()
    
    size = root.find('size')
    width = size.find('width').text
    height = size.find('height').text
    
    for child in root.findall('object'):
        name = child.find('name')
        if 'scoreboard' == name.text:
            coord = child.find('bndbox')
            xmin = coord.find('xmin').text
            xmax = coord.find('xmax').text
            ymin = coord.find('ymin').text
            ymax = coord.find('ymax').text
            
            return [xmin, xmax, ymin, ymax, width, height]

In [18]:
def create_tf_example(example):
  # TODO(user): Populate the following variables from your example.
    height = example['height'] # Image height
    width = example['width'] # Image width
    filename = example['filename'] # Filename of the image. Empty if image is not from file
    encoded_image_data = example['data'] # Encoded image bytes
    image_format = example['format'] # b'jpeg' or b'png'

    xmins = [example['xmin']] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [example['xmax']] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [example['ymin']] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [example['ymax']] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [bytes('Lichilnik', 'utf-8')] # List of string class name of bounding box (1 per box)
    classes = [1] # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    counter = 0
    for dataset in [test_images, train_images]:
        if len(dataset) > 300:
            name_writer = 'final_train_all.tfrecord'
        else:
            name_writer = 'final_test_all.tfrecord'
        writer = tf.python_io.TFRecordWriter(name_writer)


        for img in dataset:

            name = img.split('.')[0]
            print(name)
#             if os.path.isfile(folder + '/' + name + '.png'):
            if os.path.isfile(f'{folder}/{name}.png'):
                im = folder + '/' + name + '.png'
                format_img = b'png'
            elif os.path.isfile(f'{folder}/{name}.jpg'):
                im = folder + '/' + name + '.jpg'
                format_img = b'jpg'
            else:
                print(f'There is not such file: {name}')

                continue
                
            im2 = im.split('/')[1]

            data_file = folder + '/' + name + '.xml'
            if not os.path.isfile(data_file):
                print(f'No such file {data_file}')
                continue
            try:
                x_min, x_max, y_min, y_max, width, height = get_coords(data_file)
                width, height = int(width), int(height)
            except:
                continue


            with tf.gfile.GFile(im, 'rb') as fid:
                encoded_jpg = fid.read()

            if not width or not height:
                img_data_encoded = io.BytesIO(encoded_jpg)
                width, height = Image.open(img_data_encoded).size
                
            print(f"Name: {name}, x_min: {x_min}, x_max: {x_max}, y_min: {y_min}, y_max: {y_max}, width: {width}, height: {height}\n")

            exmpl = {}
            exmpl['height'] = height
            exmpl['width'] = width
            exmpl['filename'] = bytes(im, 'utf-8')
            exmpl['data'] = encoded_jpg
            exmpl['format'] = format_img
            

                
            exmpl['xmin'] = (float(x_min) - 10) / (width)
            exmpl['xmax'] = (float(x_max) + 10) / width
            exmpl['ymin'] = (float(y_min) - 10) / height
            exmpl['ymax'] = (float(y_max) + 10) / height

            print(exmpl['xmin'])

            counter += 1
            tf_example = create_tf_example(exmpl)
            writer.write(tf_example.SerializeToString())

        writer.close()
        print(f'Writed files {counter}', '\n\n')


if __name__ == '__main__':
#     tf.app.run()
    main(_)

e14
Name: e14, x_min: 755, x_max: 1523, y_min: 1224, y_max: 1471, width: 2448, height: 3264

0.3043300653594771
w166
Name: w166, x_min: 21, x_max: 207, y_min: 44, y_max: 116, width: 320, height: 240

0.034375
e144
Name: e144, x_min: 706, x_max: 1790, y_min: 1183, y_max: 1398, width: 2448, height: 3264

0.28431372549019607
g174
No such file train222/a/g174.xml
e164
Name: e164, x_min: 502, x_max: 1716, y_min: 945, y_max: 1234, width: 2448, height: 3264

0.20098039215686275
w26
Name: w26, x_min: 27, x_max: 227, y_min: 47, y_max: 97, width: 320, height: 240

0.053125
g216
Name: g216, x_min: 263, x_max: 1830, y_min: 1590, y_max: 1924, width: 2448, height: 3264

0.10334967320261437
g120
Name: g120, x_min: 689, x_max: 1483, y_min: 1333, y_max: 1542, width: 2448, height: 3264

0.27736928104575165
g46
Name: g46, x_min: 380, x_max: 1524, y_min: 1598, y_max: 1815, width: 2448, height: 3264

0.1511437908496732
e30
Name: e30, x_min: 659, x_max: 1637, y_min: 1481, y_max: 1792, width: 2448, height: 3